# Introduction

Insomnia is one of the sleeping disorder. Although we may think that insomnia can occur only on adults, in fact it also can occur on adolescents. This data science project is delving into the insomnia during the adolescent period by using a dataset which contains the records of more than 90 adolecents. Individually, all participants have been following and participating in 19 questionnaires related to the sleep pattern, sleep habit, psychological issues, etc.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv(r"../datasets/insomnia_data.csv")


# Stage 1: Data Cleaning

Based on the observations toward the dataset, it requires some data cleaning steps before the dataset is processed on the next stages. On the following code, we can see the dimension and the first five rows of the dataset.

In [3]:
df.shape

(95, 174)

In [4]:
df.head()

,ID,Group,SubGroup,Remote,Sex,Age,American_Indian,Asian,Native_Hawaiian,Black,...,Zcope_acccept,Zcope_suppression,Zcope_planning,Zders_nonaccpetance,Zders_goals,Zders_impulse,Zders_awareness,Zders_strategies,Zders_clarity,ZDERS_total
0,sub_001,0,0,0,0,19.3,0,0,0,0,...,-0.701562,2.215354,0.740137,-1.122805,1.905727,-0.114565,1.083087,-0.656051,0.538016,0.575806
1,sub_002,0,0,0,0,19.3,0,0,0,0,...,1.545013,0.327833,1.078180,-0.552396,-0.302127,-0.114565,1.083087,-0.656051,0.538016,0.153943
2,sub_003,0,0,0,1,18.8,0,0,0,0,...,-0.327133,1.743474,-0.612036,-0.267191,-1.406055,-0.425527,0.271626,-0.656051,-0.260601,-0.619473
3,sub_004,0,0,0,0,18.8,0,0,0,0,...,-1.824849,-1.559689,-0.612036,-0.552396,0.249836,-0.114565,0.596210,-0.116442,0.538016,0.224254
4,sub_005,1,2,0,1,19.6,0,0,0,0,...,-2.199279,-0.615928,-0.950080,0.588422,1.353763,0.196397,0.109334,0.153363,1.336632,0.857049


## Drop non-total features from 'Multiple Columns with Total' questionnaire type

In [8]:
# Create seven lists which contain the non-total features name that will be dropped
non_total_ASHS = ['ASHS_physiological', 'ASHS_cognitive', 'ASHS_emotional', 'ASHS_SleepEnvirnmont', 'ASHS_DaytimeSleep', 'ASHS_substances', 'ASHS_bedtimeRoutine', 'ASHS_sleepStability', 'ASHS_BedroomSharing']
non_total_GCTI = ['GCTI_anxiety', 'GCTI_reflection', 'GCTI_worries', 'GCTI_thoughts', 'GCTI_negativeAffect']
non_total_PSRS = ['PSRS_PrR', 'PSRS_RWO', 'PSRS_RSC', 'PSRS_FRa', 'PSRS_RSE']
non_total_TCQIR = ['TCQIR_Aggressive_supression', 'TCQIR_cognitive_distraction', 'TCQIR_reappraisal', 'TCQIR_behavtioral_distraction', 'TCQIR_social_avoidance', 'TCQIR_worry']
non_total_ASQ = ['asq_home', 'asq_school', 'asq_attendance', 'asq_romantic', 'asq_peer', 'asq_teacher', 'asq_future', 'asq_leisure', 'asq_finance', 'asq_responsibility']
non_total_CASQ = ['casq_sleepy', 'casq_alert']
non_total_DERS = ['ders_nonaccpetance', 'ders_goals', 'ders_impulse', 'ders_awareness', 'ders_strategies', 'ders_clarity']

# Concatenate seven lists onto a new list
non_total_cols = np.concatenate((non_total_ASHS, 
                                 non_total_GCTI, 
                                 non_total_PSRS,
                                 non_total_TCQIR,
                                 non_total_ASQ,
                                 non_total_CASQ,
                                 non_total_DERS
                                 ))
print('Amount of non-total features: {}'.format(len(non_total_cols)))
print(non_total_cols)

Amount of non-total features: 43
['ASHS_physiological' 'ASHS_cognitive' 'ASHS_emotional'
 'ASHS_SleepEnvirnmont' 'ASHS_DaytimeSleep' 'ASHS_substances'
 'ASHS_bedtimeRoutine' 'ASHS_sleepStability' 'ASHS_BedroomSharing'
 'GCTI_anxiety' 'GCTI_reflection' 'GCTI_worries' 'GCTI_thoughts'
 'GCTI_negativeAffect' 'PSRS_PrR' 'PSRS_RWO' 'PSRS_RSC' 'PSRS_FRa'
 'PSRS_RSE' 'TCQIR_Aggressive_supression' 'TCQIR_cognitive_distraction'
 'TCQIR_reappraisal' 'TCQIR_behavtioral_distraction'
 'TCQIR_social_avoidance' 'TCQIR_worry' 'asq_home' 'asq_school'
 'asq_attendance' 'asq_romantic' 'asq_peer' 'asq_teacher' 'asq_future'
 'asq_leisure' 'asq_finance' 'asq_responsibility' 'casq_sleepy'
 'casq_alert' 'ders_nonaccpetance' 'ders_goals' 'ders_impulse'
 'ders_awareness' 'ders_strategies' 'ders_clarity']


In [9]:
# Drop non-total features from the dataframe
df.drop(non_total_cols, inplace=True, axis=1)

# Current dataframe dimension after non-total features dropping
print('Current dataframe dimension after non-total features dropping: {}'.format(df.shape))

Current dataframe dimension after non-total features dropping: (95, 131)


## Drop all features start with 'Z'

In [19]:
# Get all features name
cols_name = df.columns
# Initiate 'Z' letter
first_letter = 'Z'
# Get features name start with 'Z'
z_cols = [name for name in cols_name if name[0] == first_letter]
print("Amount of features name start with 'Z': {}".format(len(z_cols)))
print(z_cols)

Amount of features name start with 'Z': 0
[]


In [20]:
# Drop features name start with 'Z'
df.drop(z_cols, inplace=True, axis=1)

# Current dataframe dimension after dropping features name start with 'Z'
print("Current dataframe dimension after dropping features name start with 'Z': {}".format(df.shape))

Current dataframe dimension after dropping features name start with 'Z': (95, 53)


In [21]:
df.head()

,ID,Group,SubGroup,Remote,Sex,Age,American_Indian,Asian,Native_Hawaiian,Black,...,cope_religion,cope_humor,cope_disengage_emo,cope_restraint,cope_socialsupp_emo,cope_acccept,cope_suppression,cope_planning,ders_total,Unnamed: 95
0,sub_001,0,0,0,0,19.3,0,0,0,0,...,6,9,4,14,14,9,13,13,87,NaN
1,sub_002,0,0,0,0,19.3,0,0,0,0,...,5,5,4,8,8,15,9,14,81,NaN
2,sub_003,0,0,0,1,18.8,0,0,0,0,...,4,9,6,6,16,10,12,9,70,NaN
3,sub_004,0,0,0,0,18.8,0,0,0,0,...,4,8,4,7,13,6,5,9,82,NaN
4,sub_005,1,2,0,1,19.6,0,0,0,0,...,4,11,4,6,9,5,7,8,91,NaN
